# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Running command git clone -q git://github.com/Theano/Theano.git /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1f0-cp36-none-any.whl size=2666379 sha256=041cb3a9e761cab5d820669c49be2cb71215f819b38d3ca8810aee93f4551093
  Stored in directory: /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-ephem-wheel-cache-pri53nb7/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1f0
    Uninstalling Theano-1.0.4+21.g8f510a1f0:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1f0
  Using cached https://files.pythonhosted.org/packages/27/8c/7608ba709bd536bc2bccb0d1abbb70aafe9cf7e0170353b4b720ed54cb71/tensorflow-1.14.0-cp36-cp36m-macosx_10_11_x86_64.whl
  Using cached http

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [13]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


Epoch 1/100
8000/8000 [==============================] - 2s 222us/step - loss: 0.4930 - acc: 0.8002
Epoch 2/100
8000/8000 [==============================] - 1s 181us/step - loss: 0.4089 - acc: 0.8222
Epoch 3/100
8000/8000 [==============================] - 1s 178us/step - loss: 0.3939 - acc: 0.8297
Epoch 4/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.3856 - acc: 0.8314
Epoch 5/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.3789 - acc: 0.8296
Epoch 6/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.3734 - acc: 0.8394
Epoch 7/100
8000/8000 [==============================] - 2s 188us/step - loss: 0.3687 - acc: 0.8481
Epoch 8/100
8000/8000 [==============================] - 2s 279us/step - loss: 0.3670 - acc: 0.8480
Epoch 9/100
8000/8000 [==============================] - 2s 277us/step - loss: 0.3627 - acc: 0.8516
Epoch 10/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.3606 - acc: 0.853

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1510   85]
 [ 196  209]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 3.2881 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 191us/step - loss: 3.2881 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 194us/step - loss: 3.2881 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 254us/step - loss: 3.2881 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 218us/step - loss: 3.2881 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 187us/step - loss: 3.2881 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 185us/step - loss: 3.2881 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 180us/step - loss: 3.2881 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 183us/step - loss: 3.2881 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 2s 190us/step - loss: 3.2881 - acc: 0.7960

In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [17]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 0.5123 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4461 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 191us/step - loss: 0.4341 - acc: 0.7956
Epoch 4/100
8000/8000 [==============================] - 1s 187us/step - loss: 0.4297 - acc: 0.7982
Epoch 5/100
8000/8000 [==============================] - 2s 190us/step - loss: 0.4280 - acc: 0.8000
Epoch 6/100
8000/8000 [==============================] - 1s 186us/step - loss: 0.4276 - acc: 0.8030
Epoch 7/100
8000/8000 [==============================] - 2s 189us/step - loss: 0.4266 - acc: 0.8042
Epoch 8/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.4271 - acc: 0.8045
Epoch 9/100
8000/8000 [==============================] - 2s 252us/step - loss: 0.4268 - acc: 0.8049
Epoch 10/100
8000/8000 [==============================] - 2s 193us/step - loss: 0.4266 - acc: 0.8056

In [19]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1547   48]
 [ 265  140]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [ ]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)